In [1]:
# from flair.models import TextClassifier
# from flair.data import Sentence
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('Flair is pretty neat!')
# classifier.predict(sentence)
# # print sentence with predicted labels
# print('Sentence above is: ', sentence.labels)

In [2]:
import pandas as pd
import numpy as np
import itertools
import codecs
import utils
import os

In [3]:
# get the current working directory
data_path = os.getcwd()+'/Datasets/'
# os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'


EUADR_temp = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)

EUADR_temp_1 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'PA']
EUADR_temp_2 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'NA']
EUADR_temp = EUADR_temp_1.append(EUADR_temp_2)
EUADR_temp.rename(columns={"SENTENCE": "text", "ASSOCIATION_TYPE": "label"}, inplace=True)

EUADR_temp.reset_index(inplace=True)

In [4]:
# Get GAD dataset

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['CLASS'] = 'gene_disease'

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['CLASS'] = 'gene_disease'

GAD_temp = GAD_target_disease_Y_N



GAD_temp['GAD_ASSOC'] = GAD_temp['GAD_ASSOC'].apply(lambda x: x.replace('Y', 'PA').replace('N', 'NA').replace('F', 'FA'))

GAD_temp.rename(columns={"GAD_ASSOC": "label", "GAD_CONCLUSION": "text"}, inplace=True)
GAD_temp.reset_index(inplace=True)

In [5]:
df = EUADR_temp[['label','text']].append(GAD_temp[['label','text']])
df.reset_index(inplace=True)

In [6]:
df.drop("index", axis=1, inplace=True)


In [7]:
# data = pd.read_csv(data_path+'CoMAGC/'+'CoMAGC.csv')

In [9]:
data_loc = '/home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/'

data = df
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv(data_loc+'train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(data_loc+'test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv(data_loc+'dev.csv', sep='\t', index = False, header = False);


In [10]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path



In [11]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_loc), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-08-28 14:59:06,321 Reading data from /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD
2019-08-28 14:59:06,323 Train: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/train.csv
2019-08-28 14:59:06,324 Dev: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/dev.csv
2019-08-28 14:59:06,325 Test: /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/test.csv


/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/home/stirunag/anaconda3/envs/flair/lib/python3.7/site-packages/flair/data_fetcher.py:463: DeprecationWarnin

In [12]:
word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast'), FlairEmbeddings('pubmed-forward'), 
FlairEmbeddings('pubmed-backward')]



2019-08-28 15:00:54,821 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/pubmed-2015-fw-lm.pt not found in cache, downloading to /tmp/tmpfuvsou6k


100%|██████████| 111081366/111081366 [00:02<00:00, 46460973.66B/s]

2019-08-28 15:00:57,402 copying /tmp/tmpfuvsou6k to cache at /home/stirunag/.flair/embeddings/pubmed-2015-fw-lm.pt


2019-08-28 15:00:57,546 removing temp file /tmp/tmpfuvsou6k
2019-08-28 15:00:57,843 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/pubmed-2015-bw-lm.pt not found in cache, downloading to /tmp/tmpdx5holkv


100%|██████████| 111081366/111081366 [00:02<00:00, 45003076.24B/s]

2019-08-28 15:01:00,519 copying /tmp/tmpdx5holkv to cache at /home/stirunag/.flair/embeddings/pubmed-2015-bw-lm.pt
2019-08-28 15:01:00,602 removing temp file /tmp/tmpdx5holkv


In [13]:
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   rnn_type = "LSTM",
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [14]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)



2019-08-28 15:01:07,453 Computing label dictionary. Progress:


100%|██████████| 2688/2688 [00:00<00:00, 142813.76it/s]

2019-08-28 15:01:07,478 [b'PA', b'NA']


In [15]:
trainer = ModelTrainer(classifier, corpus)


In [16]:
# 7. start the training
# trainer.train(data_loc, max_epochs=10)
trainer.train(data_loc,
              learning_rate=0.1,
              mini_batch_size=32,
              anneal_factor=0.5,
              patience=5,
              max_epochs=10)

2019-08-28 15:01:18,861 ----------------------------------------------------------------------------------------------------
2019-08-28 15:01:18,863 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.25, inplace=False)
          (encoder): Embedding(275, 100)
          (rnn): LSTM(100, 1024)
          (decoder): Linear(in_features=1024, out_features=275, bias=True)
        )
      )
      (list_embedding_3): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.5, inplace=Fal

2019-08-28 15:08:58,508 ----------------------------------------------------------------------------------------------------
2019-08-28 15:08:58,509 EPOCH 4 done: loss 0.5191 - lr 0.1000
2019-08-28 15:08:58,939 DEV : loss 1.0172265768051147 - score 0.628
2019-08-28 15:08:58,983 BAD EPOCHS (no improvement): 2
2019-08-28 15:08:58,984 ----------------------------------------------------------------------------------------------------
2019-08-28 15:08:59,201 epoch 5 - iter 0/84 - loss 0.83982742 - samples/sec: 1188.69
2019-08-28 15:09:00,813 epoch 5 - iter 8/84 - loss 0.52840422 - samples/sec: 160.13
2019-08-28 15:09:02,395 epoch 5 - iter 16/84 - loss 0.51962238 - samples/sec: 163.59
2019-08-28 15:09:03,927 epoch 5 - iter 24/84 - loss 0.49948559 - samples/sec: 168.66
2019-08-28 15:09:05,572 epoch 5 - iter 32/84 - loss 0.49489446 - samples/sec: 157.19
2019-08-28 15:09:07,090 epoch 5 - iter 40/84 - loss 0.49645779 - samples/sec: 170.19
2019-08-28 15:09:08,679 epoch 5 - iter 48/84 - loss 0.48

2019-08-28 15:10:47,045 ----------------------------------------------------------------------------------------------------
2019-08-28 15:10:47,047 EPOCH 10 done: loss 0.4109 - lr 0.1000
2019-08-28 15:10:47,485 DEV : loss 0.48136597871780396 - score 0.7202
2019-08-28 15:10:47,530 BAD EPOCHS (no improvement): 1
2019-08-28 15:10:50,015 ----------------------------------------------------------------------------------------------------
2019-08-28 15:10:50,017 Testing using best model ...
2019-08-28 15:10:50,019 loading file /home/stirunag/Work/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/best-model.pt
2019-08-28 15:11:20,623 0.7619	0.7619	0.7619
2019-08-28 15:11:20,623 
MICRO_AVG: acc 0.6154 - f1-score 0.7619
MACRO_AVG: acc 0.5446 - f1-score 0.68745
NA         tp: 46 - fp: 19 - fn: 61 - tn: 210 - precision: 0.7077 - recall: 0.4299 - accuracy: 0.3651 - f1-score: 0.5349
PA         tp: 210 - fp: 61 - fn: 19 - tn: 46 - precision: 0.7749 - recall: 0.9170 - accuracy: 0.72

{'test_score': 0.7619,
 'dev_score_history': [0.628,
  0.6815,
  0.5833,
  0.628,
  0.6696,
  0.8065,
  0.6488,
  0.7827,
  0.8512,
  0.7202],
 'train_loss_history': [0.5948715245439893,
  0.5668406681645484,
  0.539126393163488,
  0.5190899801396188,
  0.48838955022039865,
  0.4743121841124126,
  0.4649090130059492,
  0.4317438288458756,
  0.4263987237853663,
  0.41086282385956674],
 'dev_loss_history': [tensor(0.6471),
  tensor(0.6154),
  tensor(0.6566),
  tensor(1.0172),
  tensor(0.5822),
  tensor(0.4838),
  tensor(0.6335),
  tensor(0.4457),
  tensor(0.4117),
  tensor(0.4814)]}

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load(data_loc+'best-model.pt')


In [ ]:
sentence = Sentence('The FCRL3 polymorphisms associated with rheumatoid arthritis in a Japanese population are not associated per se with rheumatoid arthritis in a Spanish population.')
classifier.predict(sentence)
print(sentence.labels)

In [ ]:
from tqdm import tqdm

In [ ]:
ground_truth = []
predicted_label = []
for index, rows in tqdm(df.iterrows(), total=df.shape[0]):
    if rows['label'] == 'NA':
       ground_truth.append('Negative')
    else:
       ground_truth.append('Positive')
    sentence = Sentence(rows['text'])
    classifier.predict(sentence)
    predicted_label.append(sentence.labels[0].value)
    
    

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

f1_score(ground_truth, predicted_label, average='micro') 